In [354]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:2000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [348]:
import re
import numpy as np
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.cluster.util import cosine_distance
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import math


# added english stopwords
stop_words = stopwords.words('english')
MULTIPLE_WHITESPACE_PATTERN = re.compile(r"\s+", re.UNICODE)
lemmatizer = WordNetLemmatizer()

def normalize_whitespace(text):
    """
    Translates multiple whitespace into single space character.
    If there is at least one new line character chunk is replaced
    by single LF (Unix new line) character.
    """
    return MULTIPLE_WHITESPACE_PATTERN.sub(_replace_whitespace, text)


def _replace_whitespace(match):
    text = match.group()

    if "\n" in text or "\r" in text or "\r\n" in text:
        return "\n"
    else:
        return " "


def is_blank(string):
    """
    Returns `True` if string contains only white-space characters
    or is empty. Otherwise `False` is returned.
    """
    return not string or string.isspace()


def get_symmetric_matrix(matrix):
    """
    Get Symmetric matrix
    :param matrix:
    :return: matrix
    """
    return matrix + matrix.T - np.diag(matrix.diagonal())


def core_cosine_similarity(vector1, vector2):
    """
    measure cosine similarity between two vectors
    :param vector1:
    :param vector2:
    :return: 0 < cosine similarity value < 1
    """
    return 1 - cosine_distance(vector1, vector2)


def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
    
def lemmatize_text(text):
    lemmatized_text = ""
    for sentence in text:
        # tokenize the sentence and find the POS tag for each token
        pos_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))

        # we use our own pos_tagger function to make things simpler to understand.
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))

        lemmatized_sentence = []
        for word, tag in wordnet_tagged:
            if tag is None:
                # if there is no available tag, append the token as is
                lemmatized_sentence.append(word)
            else:
                # else use the tag to lemmatize the token
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
        lemmatized_sentence = " ".join(lemmatized_sentence)
        lemmatized_text += lemmatized_sentence + " "
    return lemmatized_text

class TextRank4Sentences():
    def __init__(self):
        self.damping = 0.85  # damping coefficient, usually is .85
        self.min_diff = 1e-5  # convergence threshold
        self.steps = 100  # iteration steps
        self.text_str = None
        self.sentences = None
        self.pr_vector = None

        # added for tf-idf
        self.pr_vector2 = None
        self.tfidf = TfidfVectorizer()

    def _sentence_similarity(self, sent1, sent2, stopwords=None):
        if stopwords is None:
            stopwords = []

        sent1 = [w.lower() for w in sent1]
        sent2 = [w.lower() for w in sent2]

        all_words = list(set(sent1 + sent2))

        vector1 = [0] * len(all_words)
        vector2 = [0] * len(all_words)
        
        #print(all_words)
        # build the vector for the first sentence
        for w in sent1:
            if w in stopwords:
                continue
            vector1[all_words.index(w)] += 1

        # build the vector for the second sentence
        for w in sent2:
            if w in stopwords:
                continue
            vector2[all_words.index(w)] += 1
        
        #print(vector1, vector2)
        return core_cosine_similarity(vector1, vector2)

    def _build_sent_graph(self, sentences):
        tfidf_mat = self.tfidf.fit_transform(sentences).toarray()
        
        print("문장을 벡터화 시키기 위한 단어 집합:\n",self.tfidf.get_feature_names())
        print()
        print("TF-IDF 벡터화 된 문장:\n",tfidf_mat)
        print()
        #graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        sm = np.zeros([len(sentences), len(sentences)])

        for idx1 in range(len(sentences)):
            for idx2 in range(len(sentences)):
                if idx1 == idx2:
                    continue
                sm[idx1][idx2] = core_cosine_similarity(tfidf_mat[idx1], tfidf_mat[idx2])

                
        print("TF-IDF화 된 문장 간 코사인 유사도:")
        print('    ', end=" ")
        for i in range(len(sentences)):
            print('  s%d  '%(i+1),end=" ")
        print()
        for i in range(len(sentences)):
            print('s%d'%(i+1), end=" ")
            for j in range(len(sentences)):
                print('%.5f'%sm[i][j], end= " ")
            print()   
        print()     
        
        sm = get_symmetric_matrix(sm)

        # Normalize matrix by column
        norm = np.sum(sm, axis=0)
        sm_norm = np.divide(sm, norm, where=norm != 0)  # this is to ignore the 0 element in norm

        return sm_norm
    
    def _build_similarity_matrix(self, sentences, stopwords=None):
        # create an empty similarity matrix
        sm = np.zeros([len(sentences), len(sentences)])

        for idx1 in range(len(sentences)):
            for idx2 in range(len(sentences)):
                if idx1 == idx2:
                    continue

                sm[idx1][idx2] = self._sentence_similarity(sentences[idx1], sentences[idx2], stopwords=stopwords)
        
        print("BoW 문장 간 코사인 유사도:")
        print('    ', end=" ")
        for i in range(len(sentences)):
            print('  s%d  '%(i+1),end=" ")
        print()
        for i in range(len(sentences)):
            print('s%d'%(i+1), end=" ")
            for j in range(len(sentences)):
                print('%.5f'%sm[i][j], end= " ")
            print()   
        print()
        # Get Symmeric matrix
        sm = get_symmetric_matrix(sm)
        # Normalize matrix by column
        norm = np.sum(sm, axis=0)
        sm_norm = np.divide(sm, norm, where=norm != 0)  # this is to ignore the 0 element in norm
        return sm_norm

    def _run_page_rank(self, similarity_matrix):

        pr_vector = np.array([1] * len(similarity_matrix))

        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr_vector = (1 - self.damping) + self.damping * np.matmul(similarity_matrix, pr_vector)
            if abs(previous_pr - sum(pr_vector)) < self.min_diff:
                break
            else:
                previous_pr = sum(pr_vector)
        
        return pr_vector

    def _get_sentence(self, index):

        try:
            return self.sentences[index]
        except IndexError:
            return ""

    def get_top_sentences(self, number=1):

        top_sentences = {}

        if self.pr_vector is not None:

            sorted_pr = np.argsort(self.pr_vector)
            sorted_pr = list(sorted_pr)
            sorted_pr.reverse()

            index = 0
            for epoch in range(number):
                # print(str(sorted_pr[index]) + " : " + str(self.pr_vector[sorted_pr[index]]))
                sent = self.sentences[sorted_pr[index]]
                sent = normalize_whitespace(sent)
                top_sentences[index] = sent + " : " + str(self.pr_vector[sorted_pr[index]])
                index += 1

        return top_sentences

    def get_top_sentences2(self, number=1):

        top_sentences = {}

        if self.pr_vector2 is not None:

            sorted_pr = np.argsort(self.pr_vector2)
            sorted_pr = list(sorted_pr)
            sorted_pr.reverse()

            index = 0
            for epoch in range(number):
                # print(str(sorted_pr[index]) + " : " + str(self.pr_vector[sorted_pr[index]]))
                sent = self.sentences[sorted_pr[index]]
                sent = normalize_whitespace(sent)
                top_sentences[index] = sent + " : " + str(self.pr_vector2[sorted_pr[index]])
                index += 1

        return top_sentences

    def analyze(self, text, stop_words=None, lem_flag=0):
        self.text_str = text
        self.sentences = sent_tokenize(self.text_str)

        if lem_flag == 0:
            temp = self.sentences
        else:
            temp = sent_tokenize(lemmatize_text(self.sentences))
        # 특수문자 제거
        eng_sentences = []
        for sent in temp:
            sent = re.sub('[^a-zA-Z]', ' ', sent)
            eng_sentences.append(sent)
        tokenized_sentences = [word_tokenize(sent) for sent in eng_sentences]

        # stopwords를 제거한 tokenized sentences
        rm_tokenized_sentences = []
        for sent in tokenized_sentences:
            temp = []
            for word in sent:
                if word not in stop_words:
                    temp.append(word)
            rm_tokenized_sentences.append(' '.join(temp))

        similarity_matrix = self._build_similarity_matrix(tokenized_sentences, stop_words)
        correlation_matrix = self._build_sent_graph(rm_tokenized_sentences)
        
        print("normalized sentence graph(cosine similarity):\n",similarity_matrix)
        print()
        print("normalized sentence graph(tf-idf + cosine similarity):\n",correlation_matrix)
        self.pr_vector = self._run_page_rank(similarity_matrix)
        self.pr_vector2 = self._run_page_rank(correlation_matrix)
        print()
        print("textrank vector(cosine similarity):",self.pr_vector)
        print()
        print("textrank vector(tfidf + cosine):",self.pr_vector2)
        print()

<h3>문장에 대한 유사도를 구하기 위한 함수</h3>

1. 인자로 리스트 형식의 토큰화 된 두개의 문장과 stopwords를 받아서 lower case로 문장 안의 단어들을 lower case로 만들어준다.
2. 두 문장에 포함 된 모든 단어들을 집합 객체로 구성하여 집합 객체로 만들어 중복을 제거한다.
3. 각 문장에 집합 객체에 포함된 단어가 몇개 들어 있는지 벡터로 나타내준다. stopwords에 포함되는 단어이면, continue하여 개수를 증가시키지 않는다.
4. 벡터화 된 두 문장의 cosine similarity를 계산한다.

In [349]:
def sen_sim(sent1, sent2, stopwords=None): # 토큰화 된 두개의 문장과 stopwords를 인자로 받음
        if stopwords is None:
            stopwords = []
        
        #문장을 모두 소문자로 바꿔줌
        sent1 = [w.lower() for w in sent1]
        sent2 = [w.lower() for w in sent2]
            
        all_words = list(set(sent1 + sent2))
        
        print("문장을 벡터화 시키기 위한 단어 집합:", all_words)

        vector1 = [0] * len(all_words)
        vector2 = [0] * len(all_words)

        # build the vector for the first sentence
        for w in sent1:
            if w in stopwords:
                continue
            vector1[all_words.index(w)] += 1

        # build the vector for the second sentence
        for w in sent2:
            if w in stopwords:
                continue
            vector2[all_words.index(w)] += 1
        
        print("\n1. Bow 형태로 벡터화 된 문장")
        print("벡터화된 문장 1:",vector1)
        print("벡터화된 문장 2:", vector2)
        
        v = []
        v.append(vector1)
        v.append(vector2)
        
        return v

<h3>1. 문장을 BoW 형태로 벡터화</h3>

문장 1: "The Best Italian restaurant enjoy the best pasta."<br>
문장 2: "American restaurant enjoy the best hamburger."

문장 간의 유사도를 계산하기 위해 전처리가 이루어진 문장의 벡터화가 제대로 이루어지는지 확인

예상 결과)

||restaurant|best|american|<span style="color:red">the</span>|pasta|enjoy|italian|hamburger|
|---|---|---|---|---|---|---|---|---|
|s1|1|2|0|0|1|1|1|0|
|s2|1|1|1|0|0|1|0|1|


**'the'의 경우 nltk의 불용어로 값이 단어에 해당하는 값이 0이 됨**
<hr/>
<h3>2. tf - idf 형태로 벡터화</h3>
<br>
앞서 구했던 문장 벡터와 다음의 공식을 이용하여 tf - idf 벡터를 구해준다. <br>
<h6>
<math xmlns="http://www.w3.org/1998/Math/MathML">
  <mtext>tf-idf(t,d)</mtext>
  <mo>=</mo>
  <mtext>tf(t,d)</mtext>
  <mo>&#xD7;</mo>
  <mtext>idf(t)</mtext>
</math></h6><br>

**tf란 문장에서 해당 단어가 얼마나 등장 했는지**<br>
**idf란 (전체 문서수) / (해당 단어가 나타난 문서 수)**<br><br>
scikit-learn의 경우 다음과 같은 idf 공식을 이용한다고 한다.<br>
<h6>idf = log((1+n) / (1+df(t))) + 1 </h6><br>
n: 전체 문서 수<br>
df(t): t라는 용어를 포함하는 문서의 수<br>

<hr/>
<h3>3. 코사인 유사도 계산</h3>

1) 직접 코사인 유사도 공식을 이용해 계산.<br>
2) 이후 nltk의 cosine_distance를 이용해 비교. cosine_distance의 경우 1-cosine_similarity 값이므로, 1-cosine_distance를 해준다.

In [350]:
sentence1 = "The best Italian restaurant enjoy the best pasta."
sentence2 = "American restaurant enjoy the best hamburger."

sentences = []
sentences.append(sentence1)
sentences.append(sentence2)

sentence1 = re.sub('[^a-zA-Z]', ' ', sentence1)
sentence2 = re.sub('[^a-zA-Z]', ' ', sentence2)

sentence1 = word_tokenize(sentence1)
sentence2 = word_tokenize(sentence2)

v = sen_sim(sentence1, sentence2, stop_words)

print("\n2. BoW를 이용한 코사인 유사도 계산")
print("1) 공식을 이용해 계산한 코사인 유사도 값:",np.dot(v[0],v[1])/(math.sqrt(np.dot(v[0],v[0])) * math.sqrt(np.dot(v[1],v[1]))))
print("2) nltk를 이용해 구한 코사인 유사도 값:", core_cosine_similarity(v[0],v[1]))

print("\n3. tf-idf 형태로 벡터화")
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_mat = tfidf.fit_transform(sentences).toarray()
print("문장을 벡터화 시키기 위한 단어 집합:",tfidf.get_feature_names())
print()
x = CountVectorizer(stop_words = 'english')
c_mat =x.fit_transform(sentences).toarray()
print("term-frequency vector 1:",c_mat[0])
print("term-frequency vector 2:",c_mat[1])
print()

temp = [0] * len(c_mat[0])
n = len(c_mat)

for s in c_mat:
    idx = 0
    for w in s:
        if w != 0:
            temp[idx] += 1
        idx+=1
idf_vec = [0] * len(temp)
idx = 0

for i in temp:
    idf_vec[idx] = math.log((n+1)/(i+1)) + 1
    idx += 1
    
print("idf vector:",idf_vec)
print()

temp = np.zeros((2,7))

for i in range(0, 2):
    for j in range(0, len(idf_vec)):
        temp[i][j] = (c_mat[i][j] * idf_vec[j])

norm1 = math.sqrt(np.dot(temp[0],temp[0]))
norm2 = math.sqrt(np.dot(temp[1],temp[1]))

for i in range(0, 7):
    temp[0][i] = temp[0][i] / norm1
for i in range(0, 7):
    temp[1][i] = temp[1][i] / norm2

print("1) 직접 구한 tf-idf 값 1:",temp[0])
print("2) 직접 구한 tf-idf 값 2:",temp[1])

print("3) tfidf vectorizer 값 1:",tfidf_mat[0])
print("4) tfidf vectorizer 값 2:",tfidf_mat[1])

print("\n4. tf-idf를 이용한 코사인 유사도 계산")
print("1) 공식을 이용해 계산한 코사인 유사도 값:",np.dot(tfidf_mat[0],tfidf_mat[1])/(math.sqrt(np.dot(tfidf_mat[0],tfidf_mat[0])) * math.sqrt(np.dot(tfidf_mat[1],tfidf_mat[1]))))
print("2) nltk를 이용해 구한 코사인 유사도 값:", core_cosine_similarity(tfidf_mat[0],tfidf_mat[1]))


문장을 벡터화 시키기 위한 단어 집합: ['hamburger', 'the', 'american', 'pasta', 'italian', 'restaurant', 'best', 'enjoy']

1. Bow 형태로 벡터화 된 문장
벡터화된 문장 1: [0, 0, 0, 1, 1, 1, 2, 1]
벡터화된 문장 2: [1, 0, 1, 0, 0, 1, 1, 1]

2. BoW를 이용한 코사인 유사도 계산
1) 공식을 이용해 계산한 코사인 유사도 값: 0.6324555320336759
2) nltk를 이용해 구한 코사인 유사도 값: 0.6324555320336759

3. tf-idf 형태로 벡터화
문장을 벡터화 시키기 위한 단어 집합: ['american', 'best', 'enjoy', 'hamburger', 'italian', 'pasta', 'restaurant']

term-frequency vector 1: [0 2 1 0 1 1 1]
term-frequency vector 2: [1 1 1 1 0 0 1]

idf vector: [1.4054651081081644, 1.0, 1.0, 1.4054651081081644, 1.4054651081081644, 1.4054651081081644, 1.0]

1) 직접 구한 tf-idf 값 1: [0.         0.63402146 0.31701073 0.         0.44554752 0.44554752
 0.31701073]
2) 직접 구한 tf-idf 값 2: [0.53309782 0.37930349 0.37930349 0.53309782 0.         0.
 0.37930349]
3) tfidf vectorizer 값 1: [0.         0.63402146 0.31701073 0.         0.44554752 0.44554752
 0.31701073]
4) tfidf vectorizer 값 2: [0.53309782 0.37930349 0.37930349 0.53309782 0.    

<h2>Sentence Graph와 PageRank를 이용하여 핵심 문장 추출하기</h2>

<h3>벡터화 된 문장간의 코사인 유사도를 이용하여 문장 그래프 생성</h3>

![Cormat](Downloads/cormat.jpeg)
**다음과 같은 코드를 이용해서 세 문장을 요약해본다** 

> <h3>1) BoW 문장 그래프 생성</h3>
>
>![similarity graph](Downloads/a.png)
>
> 1. 문장간의 유사도를 행렬로 나타내기 위해, 문장의 갯수 * 문장의 갯수 크기의 행렬을 생성한다.
> 2. 앞서 구현한 sentence similarity 함수를 이용하여 문장간의 sentence similarity를 그래프로 나타낸다.<br> 
> ex) <br> 
> s1: I could not solve many problems in today's test.<br>
> s2: My friends said the test was hard too.<br>
> s3: The test was very hard.
> <br>
> <br>
> s1과 s2를 벡터화 시키면 아래와 같이 나온다. <br>
> 
> ||i|not|problems|the|hard|could|said|my|was|friends|too|today|test|in|many|solve|s|
> |---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
> |s1|0|0|1|0|0|1|0|0|0|0|0|1|1|0|1|1|0|
> |s2|0|0|0|0|1|0|1|0|0|1|0|0|1|0|0|0|0|
>  
>  <br>
>
> 코사인 유사도 공식을 이용하여 두 문장의 코사인 유사도를 구하면 0.204124가 나온다. 코드로도 같은 값이 나오는지 확인해본다.<br>
> 3. 구한 코사인 유사도 행렬을 열에 대해 normalize 한(합을 1로 바꿔준) 그래프로 pagerank 알고리즘을 수행시킨다.<br>
> 4. 생각한 핵심 문장(s3)이 제대로 추출되었는지 확인한다.
>
> <h3> 2) TF-IDF 문장 그래프 생성 </h3>
>
>![tf_idf_similarity_graph](Downloads/b.png)
> <br> 문장을 벡터화 시키는 방법만 TF-IDF로 바꿔 2)와 동일한 과정을 수행한다.<br>
> 사용되는 단어 집합은 (could, friends, hard, many, my, probelms, said, solve, test, the today)
> TF-IDF값을 엑셀로 계산하여 제대로 나오는지 확인
> ![c](Downloads/c.jpeg)
>
> <h3> 3) PageRank 알고리즘을 이용한 핵심 문장 추출 </h3>
> <br> 앞서 생성한 normalize된 문장 그래프를 pagerank 알고리즘에 대입하여 textrank vector를 생성하여, 가장 높은 값을 가지는 인덱스에 해당하는 문장을 추출한다.
>
>![pagerank_alg](Downloads/d.png)
> 1. pagerank vector는 각각의 문장의 중요도에 해당하는 값을 1로 초기화 한 벡터를 앞서 구한 행렬과 damping factor(0.85)를 곱하여 계산한다.
> 2. 새로 구한 textrank vector와 계산 전의 textrank vector의 차가 min diff(1e-5) 보다 작거나, 계산을 100회 수행하였다면 멈춘다.
> 
> ![pagerank_alg](Downloads/e.png)
> 3. 계산 과정을 간략하게 따라해보면 새로 계산되는 textrank vector는 해당 sentence와 연결된 문장의 유사성과 weight에 의해 결정된다. <br>
> ex) 그림에서 볼 수 있듯이 s1의 weight은 s1과 s2사이의 유사성에 s2의 weight을 곱하고, s1과 s3사이의 유사성에 s3의 weight을 곱해 더해준 값이다. <br> <br>
> _즉 연결된 문장에 유사성이 큰 문장이 많을수록 더 중요한 문장이 되고, 중요한 문장들이 높은 유사성으로 연결되어 있을때, 그 문장이 핵심 문장이 된다._


In [351]:
test_str = """I could not solve many problems in today's test. 
My friends said the test was hard too. 
The test was very hard."""


tr4sh = TextRank4Sentences()
tr4sh.analyze(test_str, stop_words)
top_sentence = tr4sh.get_top_sentences(1)

print("BoW 방식을 이용한 핵심 문장")
print(top_sentence[0])
print()
top_sentence_tfidf = tr4sh.get_top_sentences2(1)
print("TF-IDF 방식을 이용한 핵심 문장")
print(top_sentence_tfidf[0])

BoW 문장 간 코사인 유사도:
       s1     s2     s3   
s1 0.00000 0.20412 0.28868 
s2 0.20412 0.00000 0.70711 
s3 0.28868 0.70711 0.00000 

문장을 벡터화 시키기 위한 단어 집합:
 ['could', 'friends', 'hard', 'many', 'my', 'problems', 'said', 'solve', 'test', 'the', 'today']

TF-IDF 벡터화 된 문장:
 [[0.43238509 0.         0.         0.43238509 0.         0.43238509
  0.         0.43238509 0.2553736  0.         0.43238509]
 [0.         0.50461134 0.38376993 0.         0.50461134 0.
  0.50461134 0.         0.29803159 0.         0.        ]
 [0.         0.         0.54783215 0.         0.         0.
  0.         0.         0.42544054 0.72033345 0.        ]]

TF-IDF화 된 문장 간 코사인 유사도:
       s1     s2     s3   
s1 0.00000 0.07611 0.10865 
s2 0.07611 0.00000 0.33704 
s3 0.10865 0.33704 0.00000 

normalized sentence graph(cosine similarity):
 [[0.         0.22400924 0.28989795]
 [0.41421356 0.         0.71010205]
 [0.58578644 0.77599076 0.        ]]

normalized sentence graph(tf-idf + cosine similarity):
 [[0.         0.1842

<h3> Term Frequency 방식의 한계 </h3>

s1: She looks really really pretty.<br>
s2: She is really really kind. <br>
s3: Really Really Really. <br>
s4: Sincerely Jane is so nice. <br>
s5: That is why I really love her. <br> 

1. 유사한 의미의 단어를 구별하지 못함 <br>
s2와 s4의 의미적으로 동일한 문장이지만, 문장간의 유사성이 없다고 나온다.

In [355]:
test_str = """She looks really really pretty. 
She is really really kind. 
Really Really Really.
Sincerely Jane is so nice.
That is why I really love her."""


tr4sh = TextRank4Sentences()
tr4sh.analyze(test_str, stop_words)
top_sentence = tr4sh.get_top_sentences(1)

print("BoW 방식을 이용한 핵심 문장")
print(top_sentence[0])
print()
top_sentence_tfidf = tr4sh.get_top_sentences2(1)
print("TF-IDF 방식을 이용한 핵심 문장")
print(top_sentence_tfidf[0])

BoW 문장 간 코사인 유사도:
       s1     s2     s3     s4     s5   
s1 0.00000 0.73030 0.81650 0.00000 0.57735 
s2 0.73030 0.00000 0.89443 0.00000 0.63246 
s3 0.81650 0.89443 0.00000 0.00000 0.70711 
s4 0.00000 0.00000 0.00000 0.00000 0.00000 
s5 0.57735 0.63246 0.70711 0.00000 0.00000 

문장을 벡터화 시키기 위한 단어 집합:
 ['jane', 'kind', 'looks', 'love', 'nice', 'pretty', 'really', 'she', 'sincerely', 'that']

TF-IDF 벡터화 된 문장:
 [[0.         0.         0.50504305 0.         0.         0.50504305
  0.56906489 0.40746555 0.         0.        ]
 [0.         0.58515407 0.         0.         0.         0.
  0.65933119 0.47209862 0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  1.         0.         0.         0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.         0.         0.57735027 0.        ]
 [0.         0.         0.         0.65690037 0.         0.
  0.37008621 0.         0.         0.65690037]]

TF-IDF화 된 문장 간 코사인 유사도:
       s1     s2   